In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

from tqdm import tqdm

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","1_Domain_Profiles"))
from CLASSES_DomainProfiles import DomainProfiles_Class, DomainProfiles_DataLoading_Class

In [6]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","2_Tracking_Algorithms"))
from CLASSES_TrackingAlgorithms import TrackingAlgorithms_DataLoading_Class

In [7]:
#IMPORT FUNCTIONS
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
import FUNCTIONS_Variable_Calculation
from FUNCTIONS_Variable_Calculation import *

In [8]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Tracking_Algorithms", dataName="Eulerian_CLTracking",
                                dtype='float32',codeSection = "Project_Algorithms")

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/D

In [23]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:132 



In [10]:
##############################################
#MODEL AND ALGORITHM NUMERICAL PARAMETERS

In [11]:
kms=np.argmax(ModelData.xh-ModelData.xh[0] >= 1) #finds how many x grids is 1 km
dx=int(ModelData.xh[1]-ModelData.xh[0]) #grid resolution (in km)

In [12]:
#setting convergence threshold
if ModelData.res=='1km':
    conv_thresh=1.0/1000
elif ModelData.res=="250m":
    conv_thresh=1.5/1000

In [13]:
##############################################
#DATA LOADING FUNCTIONS

In [14]:
def GetConvergence(t):
    timeString = ModelData.timeStrings[t]
    varName = 'convergence'
    convergence = CallVariable(ModelData, DataManager, timeString, varName)
    return convergence

In [15]:
##############################################
#ALGORITHM FUNCTIONS

In [16]:
#SBZ Convergence Line Search Algorithm (levels are seperate) (python version 3.10.9) (All Max Algorithm)
def FindMaxIndices(yconv):
    """find indices of local maxima above threshold."""

     #takes dconv/dx
    ddx = (yconv[1:  ] - yconv[0:-1]) / (2 * dx)
     
    #finds local max where dconv/dx sign changes
    signs = np.sign(ddx)
    signs_diff=np.diff(signs)
    local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here (it corrects the location of the derivative)
    local_maxes=local_maxes[np.where(yconv[local_maxes]>conv_thresh)] #check if convergence is greater than convergence threshold (1s-1)
    local_maxes=local_maxes[(local_maxes>50*kms)&(local_maxes<len(ModelData.xf)-50*kms)] #removes maxes that are with 50 km of y boundary
    return local_maxes

In [17]:
#Finds all local maximums (from Calculus) along each y level for a specific z level (~0.28km in this case)
def FindLocalMaxes(conv_z,t,yind, second_round=False):
    """
    #Find_Local_Maxes Function
    #(1) At a single time and z level, runs through each y-level
    #(2) At each y-level, takes the x-derivative
    #(3) Take sign(x_derivative)
    #(4) Take diff(x_derivative)
    #(5) Max is located one index to the right of where derivative changes from positive to negative or diff is +1
    #[(6) Optional: the algorithm can run a second time over the leftover maxes after removing previous maxes from temporary variable]
    """
    
    #indexes convergence in y
    yconv=conv_z[yind,:]
    
    #finds local max where dconv/dx sign changes
    local_maxes = FindMaxIndices(yconv)

    #second round maxes (not 100% necessary, only if missing many convergence maximums that are visually there)
    if second_round == True:
        yconv2=yconv.copy()
        yconv2[local_maxes]=0
        
        #takes dconv/dx
        local_maxes2 = FindMaxIndicies(f=yconv2)
    
        #combine the two
        local_maxes=np.concatenate((local_maxes,local_maxes2))
    return local_maxes

In [34]:
##############################################
#RUNNING FUNCTIONS

In [35]:
def LayerMax(t): #finds max convergence along y for multiple z location (5 is good)

    #getting convergence at time t
    conv=GetConvergence(t)
    
    #initializing data
    Nz = int(np.where(ModelData.zh <= 0.775)[0][-1]) + 1 #only include levels less than 1 km
    maxConvergence_X=np.full((Nz, ModelData.Nyh, ModelData.Nxh), -1, dtype=np.int16)
    
    #running for all z levels
    for zlev in range(Nz):
        #Taking Convergence of current timesftep
        conv_z=conv[zlev,:,:] #current z level for convergence

        for yind in range(ModelData.Nyh): #plot maximums for each row

            #finds all local maxes
            local_maxes=FindLocalMaxes(conv_z,t,yind) #convergence threshold (in 1/s)
            
            #storing data
            maxConvergence_X[zlev,yind,local_maxes] = local_maxes
    return maxConvergence_X

In [36]:
def RunAlgorithm():
    for t in tqdm(num_elements, desc="Processing timesteps"):
        # if t % 1 == 0: print(f'Processing timestep {t}/{len(data["time"])}')

        # Compute maxConvergence_X for this timestep (z,y,x)
        maxConvergence_X = LayerMax(t)

        Dictionary = {"maxConvergence_X": maxConvergence_X}

        # Directly write into HDF5 dataset at index t
        timeString = ModelData.timeStrings[t]
        TrackingAlgorithms_DataLoading_Class.SaveData(ModelData,DataManager, Dictionary, timeString)

In [37]:
#############################################
#RUNNING

In [38]:
RunAlgorithm()

Processing timesteps:   2%|▏         | 2/133 [00:00<00:14,  8.74it/s]

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_0-00-00.h5 

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_0-05-00.h5 



Processing timesteps:   3%|▎         | 4/133 [00:00<00:16,  7.75it/s]

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_0-10-00.h5 

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_0-15-00.h5 



Processing timesteps:   5%|▍         | 6/133 [00:00<00:17,  7.39it/s]

...

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_10-35-00.h5 

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_10-40-00.h5 



Processing timesteps:  98%|█████████▊| 131/133 [00:19<00:00,  7.31it/s]

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_10-45-00.h5 

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_10-50-00.h5 



Processing timesteps: 100%|██████████| 133/133 [00:20<00:00,  6.64it/s]

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_10-55-00.h5 

Saved output to /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Project_Algorithms/Tracking_Algorithms/1km_5min_34nz/Eulerian_CLTracking/Eulerian_CLTracking_1km_5min_34nz_11-00-00.h5 

